In [1]:
import pandas as pd
import gzip
import os
import numpy as np

 # A la empresa le interesa el rastreo de lo que marketing considera los productos y lo que ventas considera los mejores clientes.

## Mejores Productos
- product_id = {20001, 20002, 20003, 20004, 20005, 20006, 20007, 20009, 20011, 20032} (diez productos)

## Mejores Clientes
- customer_id = {10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10011, 10012, 10013} (doce clientes)

Total de <producto, cliente> a predecir = 10 * 12 = 120

# Objetivo
- Es el 01-enero-2020 a las 00:01 y disponibilizamos las ventas del periodo 2021912.
- El 02-enero a las 18:00 nos deben entregar:
  - El primer forecast de ventas para cada producto que se harán durante el mes 202002, de forma que nuestras plantas puedan fabricarlos durante el mes de 202001.
  - El segundo forecast es las ventas esperadas en 202002, para los 120 pares de <mejores_clientes, mejores_productos>.

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/datasets")
os.chdir("C:/diego_tools/labo3/dataset")
####################################################

In [3]:
arch_sellout = "tb_sellout_02.txt.gz"
arch_maestro_prod = "maestro_productos_depurado.csv"
arch_exogenas = "emp3_exogenas.csv"
arch_prod_ids_prediccion = "productos_a_predecir.csv"

In [4]:
# Variables para definir que atributos se descartan
meses_para_control_vigencia = [201904,201905,201906] #meses en los cuales deben aparecer los productos para ser considerados vigentes (NO discontinuados) y ser tomados en la prediccion
tope_fecha_historia = 201902 #los productos que aparezcan desde este mes (inclusive) en adelante, se excluyen por tener poca historia de ventas

In [5]:
def diferencia_meses(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

## Sellout

In [6]:
# Abrir el archivo .gz y cargarlo en un DataFrame
with gzip.open(arch_sellout, 'rt') as archivo:
    # Leer el archivo línea por línea
    df_sellout = pd.read_csv(archivo,sep="\t")

In [7]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 157.3 MB


In [8]:
# Por las dudas, eliminamos duplicados
print(len(df_sellout))
df_sellout.drop_duplicates(inplace=True)
print(len(df_sellout))

2945818
2945818


**--> sin duplicados**

In [9]:
df_sellout.isna().sum()

periodo                  0
customer_id              0
product_id               0
plan_precios_cuidados    0
cust_request_qty         0
cust_request_tn          0
tn                       0
dtype: int64

**--> sin nulos**

In [10]:
df_sellout.periodo.unique()                

array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912], dtype=int64)

In [11]:
len(df_sellout.product_id.unique())          

1233

--> algunos productos no van a tener descripción

In [12]:
len(df_sellout.customer_id.unique())            

597

In [13]:
df_sellout.plan_precios_cuidados.unique()          

array([0, 1], dtype=int64)

In [14]:
df_sellout[df_sellout.tn==0]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


--> si no hay ventas, no hay registro en 0 (directamente no hay registro)

In [15]:
df_sellout.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [16]:
# Como control, sumo tns
tn_suma_original = round(sum(df_sellout.tn))
print("Toneladas Total Control:", round(sum(df_sellout.tn),0))

Toneladas Total Control: 1324989.0


## Descarte de Productos que no hay que predecir

In [17]:
# Discontinuados (sin ventas en 3 meses mas adelante)
product_ids_vigentes =  df_sellout[df_sellout.periodo.isin(meses_para_control_vigencia)].product_id.unique()
print("Vigentes:", len(product_ids_vigentes))

product_ids_discontinuados = set(df_sellout.product_id.unique()).difference(set(product_ids_vigentes))
print("Discontinuados:", len(product_ids_discontinuados))

#Sin historia suficiente (minima fecha >= 201902)
product_ids_sin_hist_sufic = df_sellout.groupby("product_id").agg({"periodo":"min"}).reset_index()
product_ids_sin_hist_sufic = product_ids_sin_hist_sufic[product_ids_sin_hist_sufic.periodo>=tope_fecha_historia]
print(product_ids_sin_hist_sufic.head(5))
product_ids_sin_hist_sufic = product_ids_sin_hist_sufic.product_id.unique()
print("Menos 3 meses hist:", len(product_ids_sin_hist_sufic))

#Interseccion y union
print("Interseccion: ", len((set(product_ids_discontinuados).intersection(set(product_ids_sin_hist_sufic)))))
print("Union: ", len((set(product_ids_discontinuados).union(set(product_ids_sin_hist_sufic)))))


product_ids_para_predecir = set(df_sellout.product_id.unique()).difference((set(product_ids_discontinuados).union(set(product_ids_sin_hist_sufic))))
print("\nTotal:", len(product_ids_para_predecir))

Vigentes: 958
Discontinuados: 275
     product_id  periodo
31        20032   201902
126       20127   201909
173       20174   201906
208       20210   201909
211       20213   201908
Menos 3 meses hist: 182
Interseccion:  92
Union:  365

Total: 868


In [18]:
#Se guardan en un csv aparte
df_prods_prediccion = pd.DataFrame(data={"product_id":list(product_ids_para_predecir)})
df_prods_prediccion.to_csv(arch_prod_ids_prediccion, index=False)

## Completado de Períodos sin Ventas

* Los pares <cliente,producto> que no aparecen un mes van a ser completados con un registro en 0
* Sin embargo, se completará a partir de su primer mes (y no para atrás)
* Esto aplicará para el primer mes del cliente y del producto

In [19]:
periodos = df_sellout.periodo.unique()
cant_periodos = len(periodos)
periodos, cant_periodos

(array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
        201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
        201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
        201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
        201909, 201910, 201911, 201912], dtype=int64),
 36)

In [20]:
productos = df_sellout.product_id.unique()
cant_productos = len(productos)
cant_productos

1233

In [21]:
clientes = df_sellout.customer_id.unique()
cant_clientes = len(clientes)
cant_clientes

597

In [22]:
len(df_sellout),cant_productos*cant_periodos*cant_clientes

(2945818, 26499636)

**--> no todos los productos y clientes están en todos los períodos**

In [23]:
# Obtengo el primer mes de cada producto
df_primer_mes_prod = df_sellout.groupby("product_id").agg({"periodo":"min"}).reset_index()
df_primer_mes_prod = df_primer_mes_prod.rename(columns={"periodo":"primer_periodo_prod"})
df_primer_mes_prod.tail()

,product_id,primer_periodo_prod
1228,21295,201701
1229,21296,201708
1230,21297,201701
1231,21298,201708
1232,21299,201708


In [24]:
# Obtengo el primer mes de cada cliente
df_primer_mes_cliente = df_sellout.groupby("customer_id").agg({"periodo":"min"}).reset_index()
df_primer_mes_cliente = df_primer_mes_cliente.rename(columns={"periodo":"primer_periodo_cliente"})
df_primer_mes_cliente.tail()

,customer_id,primer_periodo_cliente
592,10633,201701
593,10634,201702
594,10635,201701
595,10636,201702
596,10637,201709


In [25]:
# Se va a poner 0 a todos los periodos donde el producto no se vendio
df_cartesiano = pd.DataFrame(data={"product_id":productos}).merge(pd.DataFrame(data={"periodo":periodos}), how='cross')
df_cartesiano = df_cartesiano.merge(pd.DataFrame(data={"customer_id":clientes}), how='cross')

df_cartesiano["cero_ventas"] = 0
len(df_cartesiano)

26499636

In [26]:
df_cartesiano.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26499636 entries, 0 to 26499635
Data columns (total 4 columns):
 #   Column       Dtype
---  ------       -----
 0   product_id   int64
 1   periodo      int64
 2   customer_id  int64
 3   cero_ventas  int64
dtypes: int64(4)
memory usage: 1010.9 MB


In [27]:
df_cartesiano.head()

,product_id,periodo,customer_id,cero_ventas
0,20524,201701,10234,0
1,20524,201701,10032,0
2,20524,201701,10217,0
3,20524,201701,10125,0
4,20524,201701,10012,0


In [28]:
df_sellout_complet = df_cartesiano.merge(df_sellout, how="left",on=["product_id","customer_id","periodo"])
len(df_sellout_complet)

26499636

In [29]:
df_sellout_complet.head()

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,20524,201701,10234,0,0.0,2.0,0.05300,0.05300
1,20524,201701,10032,0,0.0,1.0,0.13628,0.13628
2,20524,201701,10217,0,0.0,1.0,0.03028,0.03028
3,20524,201701,10125,0,0.0,1.0,0.02271,0.02271
4,20524,201701,10012,0,0.0,11.0,1.54452,1.54452


In [30]:
df_sellout_complet.isna().sum()

product_id                      0
periodo                         0
customer_id                     0
cero_ventas                     0
plan_precios_cuidados    23553818
cust_request_qty         23553818
cust_request_tn          23553818
tn                       23553818
dtype: int64

In [31]:
# Imputo
df_sellout_complet.cero_ventas = np.where(df_sellout_complet.tn.isna(),1,0)
df_sellout_complet.tn = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.tn)
df_sellout_complet.cust_request_tn = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.cust_request_tn)
df_sellout_complet.cust_request_qty = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.cust_request_qty)
df_sellout_complet.plan_precios_cuidados = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.plan_precios_cuidados)

In [32]:
df_sellout_complet.isna().sum()

product_id               0
periodo                  0
customer_id              0
cero_ventas              0
plan_precios_cuidados    0
cust_request_qty         0
cust_request_tn          0
tn                       0
dtype: int64

In [33]:
df_sellout_complet.cero_ventas.sum(),len(df_cartesiano)-len(df_sellout)

(23553818, 23553818)

In [34]:
# Ahora, cruzo con el primer mes y borro aquellos registros previos al primer mes de datos
print(len(df_sellout_complet))
df_sellout_complet_desde_1er_mes = df_sellout_complet.merge(df_primer_mes_prod,on="product_id",how="inner")
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes.merge(df_primer_mes_cliente,on="customer_id",how="inner")
print(len(df_sellout_complet_desde_1er_mes))

26499636
26499636


In [35]:
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes[(df_sellout_complet_desde_1er_mes.periodo >= df_sellout_complet_desde_1er_mes.primer_periodo_prod) & (df_sellout_complet_desde_1er_mes.periodo >= df_sellout_complet_desde_1er_mes.primer_periodo_cliente)]

print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes.tail(10)

19639107


,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,primer_periodo_prod,primer_periodo_cliente
26499311,20992,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499347,21018,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499383,20993,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499419,20764,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499455,21026,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499491,21054,201912,10572,1,0.0,0.0,0.0,0.0,201910,201912
26499527,20728,201912,10572,1,0.0,0.0,0.0,0.0,201911,201912
26499563,20792,201912,10572,1,0.0,0.0,0.0,0.0,201912,201912
26499599,20854,201912,10572,1,0.0,0.0,0.0,0.0,201912,201912
26499635,20770,201912,10572,1,0.0,0.0,0.0,0.0,201912,201912


In [ ]:
df_sellout_complet_desde_1er_mes['periodo_fecha'] = pd.to_datetime(df_sellout_complet_desde_1er_mes['periodo'], format='%Y%m')

# Se agrega una variable que tenga el mes y tambien la cantidad de meses de historia del producto
df_sellout_complet_desde_1er_mes["mes"] = pd.DatetimeIndex(df_sellout_complet_desde_1er_mes.periodo_fecha).month
df_sellout_complet_desde_1er_mes['primer_periodo_fecha_prod'] = pd.to_datetime(df_sellout_complet_desde_1er_mes['primer_periodo_prod'], format='%Y%m')
df_sellout_complet_desde_1er_mes["meses_historia_prod"]=df_sellout_complet_desde_1er_mes.apply(lambda row: diferencia_meses(row["periodo_fecha"],row["primer_periodo_fecha_prod"]),axis=1)
df_sellout_complet_desde_1er_mes['primer_periodo_fecha_cliente'] = pd.to_datetime(df_sellout_complet_desde_1er_mes['primer_periodo_cliente'], format='%Y%m')
df_sellout_complet_desde_1er_mes["meses_historia_cliente"]=df_sellout_complet_desde_1er_mes.apply(lambda row: diferencia_meses(row["periodo_fecha"],row["primer_periodo_fecha_cliente"]),axis=1)

# Dejo unicamente las columnas de meses_historia
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes.drop(columns=["primer_periodo_prod","primer_periodo_cliente","primer_periodo_fecha_prod","primer_periodo_fecha_cliente"])

In [ ]:
df_sellout_complet_desde_1er_mes.tail(10)

## Incorporo Maestro y Exógenas

In [ ]:
df_maestro_prod = pd.read_csv(arch_maestro_prod)

In [ ]:
df_maestro_prod.info()

In [ ]:
df_maestro_prod.head()

In [ ]:
prods_desconocidos = set(df_sellout_complet_desde_1er_mes.product_id).difference(set(df_maestro_prod.product_id))
print(len(prods_desconocidos))

In [ ]:
df_tn_prod_desc = df_sellout_complet_desde_1er_mes[df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos)]
df_tn_prod_desc.tn.sum()

In [ ]:
print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes = pd.merge(df_sellout_complet_desde_1er_mes, df_maestro_prod, on='product_id', how='left')
print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes.head()

In [ ]:
df_sellout_complet_desde_1er_mes.cat1 = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.cat1)
df_sellout_complet_desde_1er_mes.cat2 = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.cat2)
df_sellout_complet_desde_1er_mes.cat3 = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.cat3)
df_sellout_complet_desde_1er_mes.brand = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.brand)
df_sellout_complet_desde_1er_mes.sku_size = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),0,df_sellout_complet_desde_1er_mes.sku_size)
df_sellout_complet_desde_1er_mes.producto_estrella = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),0,df_sellout_complet_desde_1er_mes.producto_estrella)

In [ ]:
df_exogenas = pd.read_csv(arch_exogenas)
df_exogenas.periodo_fecha = pd.to_datetime(df_exogenas.periodo_fecha)

df_exogenas.info()

In [ ]:
# Le agregamos las exogenas
print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes = pd.merge(df_sellout_complet_desde_1er_mes,df_exogenas,on="periodo_fecha",how="left")
print(len(df_sellout_complet_desde_1er_mes))

In [ ]:
# Como control, sumo tns
print("Toneladas Total Control:", round(sum(df_sellout_complet_desde_1er_mes.tn),0),tn_suma_original==round(sum(df_sellout_complet_desde_1er_mes.tn),0))

In [ ]:
df_sellout_complet_desde_1er_mes.info()

In [ ]:
# Ordeno para que luego el FE funcione correctamente usando SHIFT
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes.sort_values(by=["product_id","customer_id","periodo"],ascending=True)

In [ ]:
df_sellout_complet_desde_1er_mes.head(50)

In [ ]:
df_sellout_complet_desde_1er_mes.isna().sum()

In [ ]:
# Exportar el DataFrame a un archivo CSV
df_sellout_complet_desde_1er_mes.to_csv("emp3_sellout_base.csv", index=False)